In [ ]:
import pandas as pd
df = pd.read_csv('./comment.csv')
df = df.rename(columns = {'0':'COMMENTS'})
df.head()

In [ ]:
#save file
df.to_csv('./comment.csv',encoding = 'utf_8_sig', index = False)

In [ ]:
data_utils.download_data_gdown("./")

In [ ]:
print("Number of comments: %i"%df.shape[0])
df.head()

In [ ]:
test1=[1,2]
test2=[2,2,2]
assert len(test1) == len(test2), 'Lists with different lengths'

## Generating wordcloud of FB comments

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

In [ ]:
#connecting all comments
comment = []
for i in range(10):
    comment.append(str(df['COMMENTS'][i]))

ws_comment = ws(comment)
pos_comment = pos(ws_comment)
entity_comment = ner(ws_comment,pos_comment)

In [ ]:
ws_comment

In [ ]:
#connecting all comments
comment = []
for i in range(len(df['COMMENTS'])):
    comment.append(str(df['COMMENTS'][i]))

ws_comment = ws(comment)
pos_comment = pos(ws_comment)
entity_comment = ner(ws_comment,pos_comment)

In [ ]:
#creating lists of ignoring words
#ignore_words = []
ignore_words
print(ignore_words)

In [ ]:
#words that should be ignore
ignore_index = []
for i in range(len(pos_comment)):
    for j in range(len(pos_comment[i])):
        if pos_comment[i][j] == 'Nh':
            ignore_index.append([i,j])
        

In [ ]:
ignore_index = ignore_index[::-1]

In [ ]:
print(ignore_index)

In [ ]:
ws_comment[3440][35]

In [ ]:
test_comment = ws(['聽'])
test_pos_comment = pos(test_comment)
print(test_pos_comment)

In [ ]:
ws_duplicate = ws_comment
ws_duplicate[3440][35]

In [ ]:

for index in ignore_index:
    if len(ws_duplicate[index[0]]) == 1:
        continue
    else:
        del ws_duplicate[index[0]][index[1]]

In [ ]:
#write the segmented words into .txt
with open('comment.txt', 'w') as c:
    for sentence in ws_duplicate:
        for word in sentence:
            try:
                c.write('%s,'%word)
            except UnicodeEncodeError:
                continue

In [ ]:
import os
from os import path
text = open('comment.txt','r').read()

In [ ]:
print(text)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
font = "C:\Windows\Fonts\mingliu.ttc"
wc = WordCloud(font_path=font,width=1200,height=600,max_font_size=180)
wc.generate(text)
wc.to_file("./wordcloud.png")
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

In [ ]:
#optional print out function
def print_word_pos_sentence(word_sentence, pos_sentence):
    #if two lists have different lenghts, raise assertion error
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
for i, sentence in enumerate(comment):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(ws_comment[i],  pos_comment[i])
    for entity in sorted(entity_comment[i]):
        print(entity)

## Post

In [ ]:
import pandas as pd
import csv
import numpy as np

import warnings
warnings.filterwarnings('ignore')

post_df = pd.read_csv('./post.csv')
post_df.head()

In [ ]:
import sys

sys.path.append(r'C:\Users\User\Anaconda3\envs\tflow\Lib\site-packages\parso\python')
sys.path

In [ ]:
from selenium import webdriver
driver = webdriver.Firefox()
driver.implicitly_wait(5)

In [ ]:
driver.get('https://zh-tw.facebook.com/')
#logging in
username = driver.find_elements_by_css_selector("input[name=email]")
username[0].send_keys('ib101301002@gmail.com')
password = driver.find_elements_by_css_selector("input[name=pass]")
password[0].send_keys('hsnu1257A')
loginButton = driver.find_elements_by_xpath('//*[contains(@id,"loginbutton")]')
loginButton[0].click()

In [ ]:
#how many likes?
like_list = []
error_list = []
so_far = 0
from time import sleep
for i in range(890,1000):
    driver.get('https://www.facebook.com/'+post_df['POST_ID'][i])

    try:
        like_number = driver.find_elements_by_xpath('//*[@class="_3dlh _3dli"]/span') #_3dlh _3dli
        like_list.append(like_number[0].text)
    except IndexError:
        try:
            like_number = driver.find_elements_by_xpath('//*[@class="_7gn0 _7gn1"]/span') # _7gn0 _7gn1
            like_list.append(like_number[0].text)
        except IndexError:
            like_list.append(None)
            error_list.append(i)
    so_far = i
'''
like_number = ''
for w in like_str:
    if w.isdigit():
        like_number = like_number + w
print(int(like_number))
'''

In [ ]:
post_df['NUM_LIKES'][890:1000] = like_list[0:110]

In [ ]:
#saving file

post_df.to_csv('./post.csv', encoding = 'utf_8_sig', index = False)

In [ ]:
#click 搜尋
driver.find_elements_by_xpath('//*[@class="crm-submit-buttons"]/span[1]/input[1]')[0].click()

In [ ]:
#click 搜尋
pager = driver.find_elements_by_xpath('//*[@class="pam uiBoxLightblue uiMorePagerPrimary"]')
pager[0].click()

In [ ]:
#get the link of like list

like_link = []
error_list = []
so_far = 0
from time import sleep
for i in range(0,10):
    driver.get('https://www.facebook.com/'+post_df['POST_ID'][i])

    try:
        likes = driver.find_elements_by_xpath('//*[@class="_3dlf"]') #normal posts
        like_link.append(likes[0].get_attribute("href"))
    except IndexError:
        try:
            likes = driver.find_elements_by_xpath('//*[@class="_7gm_"]/span') #video
            like_link.append(likes[0].get_attribute("href"))
        except IndexError:
            like_link.append(None)
            error_list.append(i)
    so_far = i

In [ ]:
#in likes page: clicking "more" until there is no "more"
import time

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    last_height = driver.execute_script("return document.body.scrollHeight")
    sleep(0.5)
    try:
        more= driver.find_elements_by_xpath('//*[@class="pam uiBoxLightblue uiMorePagerPrimary"]')
        more[0].click()
    except IndexError:
        break

In [ ]:
#extract all the proflie links
user_list = []
profiles = driver.find_elements_by_xpath('//*[contains(@class,"_5j0e fsl fwb fcb")]/a[1]')
for profile in profiles:
    user_list.append(profile.get_attribute("href"))

In [ ]:
#delete duplicate profile links

import re

user_set = set()
for user in user_list:
    text = user
    m = re.search('(.+?)\&', text)
    user_set.add(m.group(1))

In [ ]:
#given the profile links, get the links to anything
user_list = list(user_set)
driver.get(user_list[1]+'&sk=likes')

In [ ]:
#get liked pages for all the users

scroll_time = 10
pages_list = []
user_list = list(user_set)
for i in range(len(user_list)):
    individual_pages = []
    text = user_list[i]
    like_page = text +'&sk=likes'
    driver.get(like_page)
    for i in range(scroll_time):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        last_height = driver.execute_script("return document.body.scrollHeight")
        sleep(1)
        
    pages = driver.find_elements_by_xpath('//*[contains(@class,"fsl fwb fcb")]')
    for page in pages:
        try:
            individual_pages.append(page.text)
        except AttributeError:
            individual_pages.append(page[0].text)
    pages_list.append(individual_pages)

In [ ]:
pages_list

In [ ]:
pages_dic = {}
for pages in pages_list:
    for page in pages:
        if page in pages_dic.keys():
            pages_dic[page] += 1
        else:
            pages_dic[page] = 1

In [ ]:
x = pages_dic.copy()
sorted_x = sorted(x.items(), key=lambda kv: kv[1])

In [ ]:
sorted_x.reverse()
sorted_x


# Categorizing posts

In [ ]:
groups = {"言論自由":['言論','集社','自由'],\
          "LGBTI":['LGBT','LGBTI','同性','性別','兩性'],\
          "生存權":['難民','武裝','戰爭'],\
          "環境土地":['氣候','暖化'],\
          "中國人權":['中國','香港'],\
          "廢除死刑":['廢死','死刑']
         }

# Scraping ads posts info

In [ ]:
import pandas as pd
import csv
import numpy as np

import warnings
warnings.filterwarnings('ignore')

post_df = pd.read_csv('./0926AdsPosts.csv')
post_df.head()

In [ ]:
import sys

sys.path.append(r'C:\Users\User\Anaconda3\envs\tflow\Lib\site-packages\parso\python')
sys.path

In [ ]:
#open a new web browser
from selenium import webdriver
driver = webdriver.Firefox()
driver.implicitly_wait(2)

In [ ]:
driver.get('https://zh-tw.facebook.com/')
#logging in
username = driver.find_elements_by_css_selector("input[name=email]")
username[0].send_keys('ib101301002@gmail.com')
password = driver.find_elements_by_css_selector("input[name=pass]")
password[0].send_keys('hsnu1257A')
loginButton = driver.find_elements_by_xpath('//*[contains(@name,"login")]')
loginButton[0].click()

In [ ]:
#go to Facebook business manager
driver.get('https://business.facebook.com/adsmanager/pages?act=193515561207372&business_id=193512787874316')

In [ ]:
touched_number = driver.find_elements_by_xpath('//*[@class="_qea"]')
touched_number[0].text

In [ ]:
#see if an ad post has touched our customers
from time import sleep

for i in range(1275,len(post_df['ADS_ID'])):
    search = driver.find_elements_by_xpath('//*[@placeholder="Search..."]')
    search[0].send_keys(str(post_df['ADS_ID'][i]))
    sleep(0.5)
    search_button = driver.find_elements_by_xpath('//*[@class=" _t7v"]')
    search_button[0].click()
    sleep(3)
    touched_number = driver.find_elements_by_xpath('//*[@class="_qea"]')
    try:
        int(touched_number[0].text)
        post_df['PUBLISHED'][i] = 0
    except:
        post_df['PUBLISHED'][i] = 1
    #clear search
    sleep(1)
    clear_button = driver.find_elements_by_xpath('//*[@class="img sp_TiTCV7RWws__1_5x sx_52b51e"]')
    clear_button[0].click()
    sleep(1)
    try:
        clear_button = driver.find_elements_by_xpath('//*[@class="img sp_TiTCV7RWws__1_5x sx_52b51e"]')
        clear_button[0].click()
    except:
        continue

In [ ]:
post_df.to_csv('./0926AdsPosts.csv',encoding = 'big5', index = False)

## See who shared our posts

In [ ]:
import pandas as pd
import csv
import numpy as np

import warnings
warnings.filterwarnings('ignore')

post_df = pd.read_csv('./0926AdsPosts.csv')
post_df.head()

In [ ]:
from selenium import webdriver
driver = webdriver.Firefox()
driver.implicitly_wait(2)

In [ ]:
driver.get('https://zh-tw.facebook.com/')
#logging in
username = driver.find_elements_by_css_selector("input[name=email]")
username[0].send_keys('ib101301002@gmail.com')
password = driver.find_elements_by_css_selector("input[name=pass]")
password[0].send_keys('hsnu1257A')
loginButton = driver.find_elements_by_xpath('//*[contains(@name,"login")]')
loginButton[0].click()

In [ ]:
share = driver.find_elements_by_xpath('//*[@class="_355t _6iik"]')
share[0].click()
sleep(0.5)

In [89]:
from time import sleep
import re

shared_dict = {}
for i in range(959,1500):
    if post_df['PUBLISHED'][i] == 1:
        driver.get('https://www.facebook.com/'+str(post_df['ADS_ID'][i]))
        sleep(3)
        shared_dict[str(post_df['ADS_ID'][i])] = []
        #open the list of shares
        try:
            share = driver.find_elements_by_xpath('//*[@class="_355t _4vn2"]')
            share[0].click()
            sleep(2)
        except:
            try:
                share = driver.find_elements_by_xpath('//*[@class="_355t _6iik"]')
                share[0].click()
                sleep(2)
            except:
                continue
        #share[0].click()
        j = 0
        while j < 100000:
            height = j*250
            driver.execute_script("window.scrollTo(0, %s);"%height)
            #last_height = driver.execute_script("return document.body.scrollHeight")
            sleep(1)
            try:
                #get profile links
                profile_link = driver.find_elements_by_xpath('//*[@class="profileLink"]')
                text = str(profile_link[j].get_attribute("href"))
                shared_dict[str(post_df['ADS_ID'][i])].append(text)
                j+=1
            except IndexError:
                break

    else:
        continue

KeyError: 1470

In [ ]:
i

In [ ]:
shared_dict['2679389648843301']

In [90]:
#write files

import csv
csv_file = "ads_shared.csv"
#use 'w' if you want to start writing a new file
#use 'a' if you want to append data to existing file
with open('ads_shared.csv', 'a') as f: 
    for key in shared_dict.keys():
        for content in shared_dict[key]:
            f.write("%s,%s\n"%(key,content))

# Functions

In [ ]:
import re
def get_profile_link(url):    
    text = str(url)
    m = re.search('(.+?)\&', url)
    if m == None:
        return text
    return m

def access_like_page(profile_link):
    return str(profile_link)+'&sk=likes'